In [1]:
#pip install spotipy --upgrade              # Uncomment this and run it if you haven't installed spotipy before


In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy                          # Set up a config file with ckey and skey. These are available if you go
from spotipy.oauth2 import SpotifyClientCredentials      # to https://developer.spotify.com/, click on Dashboard 
                                        # from the horizontal black menu, login with your normal user info. Click 
# Import Keys                           # on "create an app" if you haven't yet, it doesn't matter what you call it.
from config import clientID, clientSEC           # Then click into
                                        # your project and you should see Client ID and Client Secret. Those are your 
                                        # ckey and skey. 

In [3]:
# Setting up Spotify API info

client_credentials_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSEC)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

1. Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there. 
2. Go to https://spotifycharts.com
2. Choose the country you want to work on. 
3. Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up. 

In [4]:
# Create dataframe from weekly chart data


path = r"input_files/spotify_top200_charts/*.csv"                # The path requires "".csv" at end of file name. 
                                                                 # This is to prevent the program from blowing up
                                                                 # when it hits some kind of hidden file. 

country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1)    # Sets up **main dataframe** with data from **FIRST** file
string = str(glob.glob(path)[0])                                 # in the folder.
year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)   # This line extracts the year and month from the 
                                                                 # **file name**
country_tracks_df[['Year']] = year_month[0]
country_tracks_df[['Month']] = year_month[1]
country_tracks_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1]) 
Tot_Streams1 = country_tracks_df['Streams'].sum()                # Find out total streams in FIRST file in folder.
country_tracks_df[['Stream %']] = country_tracks_df['Streams'] / Tot_Streams1    # New column with % of streams

for file in glob.glob(path)[1:]:                                 # Now that you have the dataframe set up from the 
        temp_df = pd.read_csv(file, header=1)                    # first file in the folder, this iterates through 
        string = str(file)                                       # remaining files
        year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        #print (year_month)
        Tot_Streams2 = temp_df['Streams'].sum()
        temp_df[['Year']] = year_month[0]
        temp_df[['Month']] = year_month[1]
        temp_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1])
        temp_df[['Stream %']] = temp_df['Streams'] / Tot_Streams2
        country_tracks_df = pd.concat([country_tracks_df,        # Adds temperary datafame to end of main dataframe
                                       temp_df])                 # as new rows.  ¿¿¿???? 
        

                                                 
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])  # Sort the new dataframe by year and month




                                                                 # You should get 4,800 rows (24 months x 200 
                                                                 # tracks per month)
country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,yyyy-mm,Stream %
0,1,TVTB (feat. Dark Polo Gang),Fedez,2907906,https://open.spotify.com/track/3J1ZGGnDdhfAN3z...,2019,01,2019-01,0.027133
1,2,Stamm Fort (feat. Sfera Ebbasta),Luche,2601065,https://open.spotify.com/track/7iTTQvztpfrDneV...,2019,01,2019-01,0.024270
2,3,È sempre bello,Coez,2360029,https://open.spotify.com/track/18LXrvm337xqvFx...,2019,01,2019-01,0.022021
3,4,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2220963,https://open.spotify.com/track/2fkVZYFv9hOejIs...,2019,01,2019-01,0.020724
4,5,Calma - Remix,Pedro Capó,2039849,https://open.spotify.com/track/5iwz1NiezX7WWjn...,2019,01,2019-01,0.019034
...,...,...,...,...,...,...,...,...,...
195,196,SICKO MODE,Travis Scott,226140,https://open.spotify.com/track/2xLMifQCjDGFmkH...,2020,12,2020-12,0.002623
196,197,Lacri-ma,Gazzelle,225380,https://open.spotify.com/track/0N6QZADGv1zv3Jb...,2020,12,2020-12,0.002615
197,198,Shallow,Lady Gaga,224420,https://open.spotify.com/track/2VxeLyX666F8uXC...,2020,12,2020-12,0.002603
198,199,"Alright (feat. Emis Killa, Geolier)",Gemitaiz,224156,https://open.spotify.com/track/7hhg8fAx3sc2zOO...,2020,12,2020-12,0.002600


In [ ]:
# Get Track IDs
track_names = country_tracks_df['Track Name'].to_list()           # Set up list of tracks to iterate through         
track_ids = []                                                    # Empty list to record track IDs into

for track in track_names:                                         # Heads up: with 4800 tracks to process, this takes
    song_results = sp.search(q=track, type='track', limit=1)      # awhile 
    try:
        track_ids.append(song_results['tracks']['items'][0]['id']) # Prevents program from blowing up - few tracks
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   # Just to let you know it's working
    except IndexError:                                             # lack track ids
        track_ids.append(np.nan)                                   # nan if nothing

TVTB (feat. Dark Polo Gang) song ID : 3J1ZGGnDdhfAN3z4qRRonq
Stamm Fort (feat. Sfera Ebbasta) song ID : 7iTTQvztpfrDneVSX9IGBD
È sempre bello song ID : 6IdiuMw1FSAvU3e6bgWQVX
IL CIELO NELLA STANZA (feat. NSTASIA) song ID : 2fkVZYFv9hOejIsLzZy8ad
Calma - Remix song ID : 5iwz1NiezX7WWjnCgY5TH4
Happy Birthday song ID : 2bnTgPFD75vbq1AhxLJfMu
7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Baila Baila Baila song ID : 7mWFF4gPADjTQjC97CgFVt
90MIN song ID : 3Ls9yKzshQye18Tx2AvOb6
Hola (I Say) (feat. Tom Walker) song ID : 42LrSpEHmXHfjEH8NeiQQi
Cambiare Adesso song ID : 2xjxaKiYssIdDU6DCPpZPt
Torna a casa song ID : 3590AAEoqH50z4UmhMIY85
Sweet but Psycho song ID : 7DnAm9FOTWE3cUvso43HhI
Ngopp'A Luna song ID : 2CfWwaXsDgNxBE4lTJZC0A
Cascare nei tuoi occhi song ID : 60oJSn5MTG2VNvCJKwd65M
Holding out for You (feat. Zara Larsson) song ID : 6eiVP1Q0XaevZEZnQFpYEH
Senza Di Me (feat. Venerus & Franco126) song ID : 3xGD5KhnizETt7KQwxtUAc
Stanza Singola (feat. Tommaso Paradiso) song ID : 23hpOtox5sIuRqVBVVJ

Be Alright song ID : 3EPXxR3ImUwfayaurPi3cm
scuol4 song ID : 1V7hvLe3fpZTMoqKLkBDgW
Uh Ah Hey song ID : 2zsJ7dZdvtmUBDOuRSqQ5h
Figurati noi (feat. Emma Muscat) song ID : 5AQ7Wuc1UGCROIifaInqdV
IDGAF song ID : 76cy1WJvNGJTj78UqeA5zr
Purple Rain (feat. Gemitaiz & Frenetik&Orang3) song ID : 286i6Yq99ALCLHXDofuTVL
Pesto song ID : 4UHDgfDtSSv9SqyfEFr4sF
Say My Name song ID : 7H6ev70Weq6DdpZyyTmUXk
Buongiornissimo song ID : 2vmjhinZGxfcN5KOkFnGYQ
L'altra dimensione song ID : 36ONZ2CvV8KsLPapNPGTqa
Chic song ID : 4wVOKKEHUJxHCFFNUWDn0B
Solo (feat. Demi Lovato) song ID : 6kPJZM97LwdG9QIsT7khp6
Irraggiungibile (feat. Federica Carta) song ID : 03PDOt4cRKQgpYfA9I9wnC
Don't Stop Me Now - Remastered 2011 song ID : 7hQJA50XrCWABAu5v6QZ4i
These Days (feat. Jess Glynne, Macklemore & Dan Caplen) song ID : 5CLGzJsGqhCEECcpnFQA8x
PELLE D'OCA song ID : 7BIoYqwZlcKosZr1ceAhuV
Rollin' song ID : 1xqRKo0SDFDO5Jy8yEVD0X
Un posto bellissimo song ID : 2qqDDnzVeEk4Yf0rJPaM9h
Che Ci Faccio song ID : 3yhRpGAdByLRLN

Peter Pan - Vuoi volare con me? song ID : 3qbMJGhSNdeKdz407PA6x4
Sopra song ID : 5d5hgQCHyzkPrxWejb69St
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Nothing Breaks Like a Heart (feat. Miley Cyrus) song ID : 27rdGxbavYJeBphck5MZAF
Ludovico song ID : 1BncfTJAWxrsxyT9culBrj
Let Me Down Slowly song ID : 2qxmye6gAegTMjLKEBoR3d
Musica che resta song ID : 59nLT8Be6wKKhgfJKkLN0j
Racconterò di te song ID : 1c1f9Xtqp0dwUkXYndACqe
PIOGGIA song ID : 2unmie5sfHDGwSQO9jEok4
Better Now song ID : 7dt6x5M1jzdTEt8oCbisTK
Gli occhi della tigre song ID : 5pL8J6weoydzXglhsoJ6Vw
Thunderclouds song ID : 4lJNen4SMTIJMahALc3DcB
Se Piovesse Il Tuo Nome song ID : 0FNRyMufs8Ox03TtaoyCjH
Money Rain song ID : 1kuSXR7D6H02EsHcK4ULFD
Beatrix Kiddo (feat. Vale Lambo) song ID : 3NR8xopBopMRrl8yA3dOyI
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Another One Bites The Dust - Remastered 2011 song ID : 57JVGBtBLCfHw2muk5416J
Nuovi Jeans song ID : 0N1pKkjhVwKZWFLF47R1tv
Fame song ID : 1PehfI

Vertigini song ID : 1Oou7m2VuxCDSOdqsu07TU
Tesla (feat. Sfera Ebbasta & DrefGold) song ID : 093RgZ7RXHZwIrxZhZYX4Q
Sunset Lover song ID : 0hNduWmlWmEmuwEFcYvRu1
Davide (feat. Coez) song ID : 1Lq5ApqhIWUXzlryFX2K6J
I Can’t Get Enough (benny blanco, Selena Gomez, J Balvin, Tainy) song ID : 1WSGTYKLEpKfNQYYW5Oymt
Don't Call Me Up song ID : 5WHTFyqSii0lmT9R21abT8
Gang Shit (feat. Capo Plaza) song ID : 7EkJMf7yOPU7wOK2FcryQg
Ti dedico il silenzio song ID : 6GLmGfALwjzCaOiHwgfU7g
Close To Me (with Diplo) (feat. Swae Lee) song ID : 5JEx7HbmvHQQswJCsoo9rA
Porto Cervo song ID : 0TAt3c8KXb1D3sWT8ENar8
Cosa ti aspetti da me song ID : 0zTTbiHlCRzrA5O9F6Vnem
Non Abbiamo Età song ID : 0NwuXciw6eQc4edjwSBnV3
Habitué - Italian Remix song ID : 3z50RSsxr4gvmTu8MXmoKv
Say My Name song ID : 7H6ev70Weq6DdpZyyTmUXk
Te Robaré song ID : 4PJ3umtW1aWdfh5EITHc6T
Certi Giorni (feat. Nitro) song ID : 4nvsA575k8HrEjdQ78Ymvi
Toradol song ID : 3Rmv3h4ovgt9xVgWOfMlLx
Dancing With A Stranger (with Normani) song ID : 6Q

Old Town Road song ID : 2YpeDb67231RjR0MgVLzsG
90MIN song ID : 3Ls9yKzshQye18Tx2AvOb6
2% (feat. Frah Quintale) song ID : 5ODlTG5qxhCsC05E36Rv25
High Hopes song ID : 1rqqCSm0Qe4I9rUvWncaom
Posso (feat. Max Gazzé) song ID : 5N0nmOmc0MqPZTYKw4fVV6
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
bury a friend song ID : 4SSnFejRGlZikf02HLewEF
Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
Happier song ID : 2dpaYNEQHiRxtZbfNsse99
SOS song ID : 30cSNer6TV8x2utjULVeQ5
Senza Di Me (feat. Venerus & Franco126) song ID : 3xGD5KhnizETt7KQwxtUAc
Secreto song ID : 5W83ErFkO3aKAIS1WMi6u0
C'est la vie song ID : 4RPpx2lgTptIUtR50KXfQ0
7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Happy Birthday song ID : 2bnTgPFD75vbq1AhxLJfMu
Old Town Road (feat. Billy Ray Cyrus) - Remix song ID : 6zpiwiexX8HpFCZLyIWKDv
Anche Fragile song ID : 05G23JkXHRv3NJ9hENaafi
Domenica song ID : 1blcymJaZW6NSHTfFD95WB
CABRIOLET (feat. Sfera Ebbasta) song ID : 7wC76LWyNxaP3hjGzbWOcN
Fuoco e benzina song ID : 4Fs1sEg9Mwn6vykFD89xvc


...

Playa song ID : 243jb092tx4s5UZntX2xhC
OPS song ID : 59fFsGRwFiuhxKDPc1k6xW
El Efecto song ID : 4SJHHpKhcuXwDsCsPqshXj
Muévelo song ID : 4VgYtXCVJ7IbWAZ5ryfvEQ
Alone, Pt. II song ID : 0bMbDctzMmTyK2j74j3nF3
LOVE BANDANA song ID : 65bM8F5KQPK4idHTJJbZEA
Veleno 7 song ID : 2jkIbZ43eH3rM1yro2Ojzh
On Gang song ID : 6naTU6BOOgi9FnXnqjmrE2
Twerk (con Boro Boro) song ID : 4jiCUUfjuJoG03gnw3jj1r
Il tuo profumo song ID : 0sqbGUEecNagMilnntIsuZ
Regina Coeli song ID : 7wp6HWS8VBbWedWcrxThjN
La Mia Hit (feat. Max Pezzali) song ID : 4HwSZhAc4Gvvvbjj83TJDz
Chic song ID : 4wVOKKEHUJxHCFFNUWDn0B
I Don't Care (with Justin Bieber) song ID : 3HVWdVOQ0ZA45FuZGSfvns
Mondo song ID : 5WIQTUDLCEY1g1lmOWvDB4
Intentions (feat. Quavo) song ID : 4umIPjkehX1r7uhmGvXiSV
Senza Di Me (feat. Venerus & Franco126) song ID : 3xGD5KhnizETt7KQwxtUAc
Supercalifragili (feat. Annalisa & Luca Di Stefano) song ID : 0pwbULuHR9ufNE5FhOGcCQ
Pianeti song ID : 2w5wtWq30yTkNSG9YjMFvW
Rondini Al Guinzaglio song ID : 1jhHPnmgZepKU8qX4V

La hit dell'estate song ID : 443cnFF139Ql85enXOTWEu
Margarita (con Marracash) song ID : 3qBtltssunsFuzn6DxXI0b
Supalonely song ID : 4nK5YrxbMGZstTLbvj6Gxw
Calipso (with Dardust) (feat. Sfera Ebbasta, Mahmood & Fabri Fibra) song ID : 45zJRuzK9e6AO527NOW7Fy
223 song ID : 4sjiIpEv617LDXaidKioOI
Stu core t'apparten song ID : 3nhAgjyrfUUCNDMZHx6LCa
IL CIELO NELLA STANZA (feat. NSTASIA) song ID : 2fkVZYFv9hOejIsLzZy8ad
Cracovia, Pt. 3 song ID : 2dS1EkNtCC6TnYNdsfin08
PAROLA - feat. Lazza, Emis Killa song ID : 2tQ7vPjN88wkQl5hSNSbVF
Guaranà song ID : 1KXpcoggYmNTPCRXIz2m9Z
Someone You Loved song ID : 7qEHsqek33rTcFNT9PFqLf
Fuego (with Geolier & Lele Blade) - Prod. Poison Beatz, Tom Trigger song ID : 6XPYGFacwZAtmoSVi9m4eT
Breaking Me song ID : 3H7ihDc1dqLriiWXwsc2po
CHANCE song ID : 5u1n1kITHCxxp8twBcZxWy
Stuck with U (with Justin Bieber) song ID : 4HBZA5flZLE435QTztThqH
BIBERON (feat. DrefGold & ANNA) song ID : 7ldYwTP1H5HpzJ3Oe3LN5G
Bando - Remix song ID : 1Upi1oHB1ly7XdlbOz7Gjd
Non Avere P

Blinding Lights song ID : 0VjIjW4GlUZAMYd2vXMi3b
ily (i love you baby) (feat. Emilee) song ID : 62aP9fBQKYKxi7PDXwcUAS
Toosie Slide song ID : 127QTOFJsJQp5LbJbu3A1y
Autostop song ID : 5ZgeeXOHGYUn0K3kuICyw9
Dilemme (with tha Supreme & Mara Sattei) song ID : 1Whf2ZaesM1gXlU0z3Fslt
16 Marzo (feat. Gow Tribe) song ID : 2kZV7d0A4BIN5I8iFBnbqp
NEON - Le Ali (feat. Elisa) song ID : 76cEI7LczlKZ1yLAHIoPBJ
Missili song ID : 5dcyltc1ataieXIeTqhH3t
Rain On Me (with Ariana Grande) song ID : 24ySl2hOPGCDcxBxFIqWBu
Lo Sai song ID : 2fQtrzMUMmnnJ5YxSb55MR
In Your Eyes song ID : 7szuecWAPwGoV1e5vGu8tl
Baila Conmigo (feat. Kelly Ruiz) song ID : 4Clmg1g8n2yBYG13Z1oSdG
Boss Bitch song ID : 78qd8dvwea0Gosb6Fe6j3k
Party HH - feat. Lazza song ID : 61koTDrwN9kZQmDhtH7NpM
Per sentirmi vivo song ID : 20v9X8TV0AgwNGFT71OX3w
blun7 a swishland song ID : 7HwvPmK74MBRDhCIyMXReP
Break My Heart song ID : 017PF4Q3l4DBUiWoXk4OWT
TKN (feat. Travis Scott) song ID : 4w47S36wQGBhGg073q3nt7
Te lo prometto song ID : 25WI6xo

Mediterranea song ID : 0kN8aoF74sj0UCFwMQoIBK
Karaoke song ID : 4nefFqiukTvjgt8hkv73PP
M' Manc (con Geolier & Sfera Ebbasta) song ID : 0MLu1cFYL4ikdRTdasCQT9
Non mi basta più (feat. Chiara Ferragni) song ID : 5ZUeBvBEH8hpdLMW3yCmHb
Paloma (feat. Anitta) song ID : 0HFkCSXRTUHCYJ148PdD0v
Bimbi Per Strada (Children) song ID : 6KF78W66DcQ9m5YqR0r5Ig
Superclassico song ID : 6Ahg1hncxUdK0ICqU03BCu
Sono un bravo ragazzo un po' fuori di testa song ID : 5EXLYWZGnmYC01H53WUAhi
Savage Love (Laxed - Siren Beat) song ID : 1xQ6trAsedVPCdbtDAmk0c
Breaking Me song ID : 3H7ihDc1dqLriiWXwsc2po
MAMACITA song ID : 14wf185UxfNbSy8dwt4r4q
Autostop song ID : 5ZgeeXOHGYUn0K3kuICyw9
Spigoli song ID : 1XQvMjA9IiIx1lAiTHSxQi
Good Times song ID : 0QwZfbw26QeUoIy82Z2jYp
Chico (feat. Rose Villain & Luchè) song ID : 3Xhl7OqWAFgVxQKrWhyVVO
Guaranà song ID : 1KXpcoggYmNTPCRXIz2m9Z
Ciclone (feat. Elodie, Mariah, Gipsy Kings, Nicolás Reyes, Tonino Baliardo) song ID : 2YGqtxaGdML9aM5jCwgRw9
2 TIRI (feat. tha Supreme) son

Morto Dentro song ID : 1H9PaGk1HDQGE4Zl9vDCNf
Don't Start Now song ID : 6WrI0LAC5M1Rw2MnX2ZvEg
Stu core t'apparten song ID : 3nhAgjyrfUUCNDMZHx6LCa
Nera song ID : 6Lc0La3OwOmAouQjbID4uX
I Wanna Be an Illusion song ID : 0BhDhjb88sveB3rWJTx9wv
IL CIELO NELLA STANZA (feat. NSTASIA) song ID : 2fkVZYFv9hOejIsLzZy8ad
Polo nord - C+N Progressive Mix song ID : 1Wbc0N0X1zkFqnKGcj8Quh
Bando song ID : 4BxzNZpcm1j3JczZzGNe2k
Playa song ID : 243jb092tx4s5UZntX2xhC
Dilemme (with tha Supreme & Mara Sattei) song ID : 1Whf2ZaesM1gXlU0z3Fslt
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Ferma A Guardare song ID : 1eDC4NiUYgQSKpKDIvXxi4
Beretta song ID : 0lB24x4SNbi8lLhUAsdoMO
AMIRI BOYS (feat. Capo Plaza) song ID : 1ioNhImsVeB6VaLK6kBN97
Non spegnere la musica song ID : 3HgDojnZf20hPOsuPRyGNU
#THOTMILANODM song ID : 4wz9be3iTHln8swogVIAwR
Oh Cacchio song ID : 10y452DRiLSzDQAcX2Ek2j
HIGHEST IN THE ROOM song ID : 3eekarcy7kvN4yt5ZFzltW
SCACCIACANI song ID : 2FbPSLBODnLZO1Gou746vW
Riccione song ID : 6RiprjTT

TIP: for this next section, add an "output_files" subfolder to export into. 

In [ ]:
# Add Track IDs to dataframe

country_tracks_df['Track ID'] = track_ids                  # Add new column with track IDs

In [ ]:
# Drop empty songs and export dataframe to csv to back it up

clean_country_tracks_df = country_tracks_df.dropna(how='any')           # Use .dropna() to remove rows with missing data

clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  # Back up to .csv

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")

country_track_ids_df

In [ ]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
tempo = []

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        tempo.append(feat_results[0]['tempo'])
        print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        tempo.append(np.nan)
        

In [ ]:
# Add audio features to dataframe

country_track_ids_df['Danceability'] = danceability     # Add new columns with audio features
country_track_ids_df['Valence'] = valence
country_track_ids_df['Energy'] = energy
country_track_ids_df['Tempo'] = tempo

# Add new columns with product of % and each feature
country_track_ids_df['Danceability_Stream%'] = danceability * country_track_ids_df['Stream %']    
country_track_ids_df['Valence_Stream%'] = energy * country_track_ids_df['Stream %'] 
country_track_ids_df['Energy_Stream%'] = valence * country_track_ids_df['Stream %'] 
country_track_ids_df['Tempo_Stream%'] = tempo * country_track_ids_df['Stream %'] 

In [ ]:
# Back up dataframe again to .csv
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')          # Use .dropna() to remove rows with missing data

clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) #Back up the dataframe to csv again

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

In [ ]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence_Stream%'].sum()

country_tracks_data_groupby

In [ ]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [ ]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH')         # Set up main title

y_axis = valence_2019['Valence_Stream%']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence_Stream%'], label="2019 Weighted Avg Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence_Stream%'], label="2020 Weighted Avg Valence", marker='o', color='steelblue')  # Plot 2020

plt.xlabel('Months')                                                 # Set up axis titles
plt.ylabel('Valence Score')

plt.xlim(-0.75, len(x_axis)-0.25)                                    # Set up axis limits
plt.ylim(0, 1)

plt.legend()                                                         # Include the legend

plt.show()


In [ ]:
# Compare valence scores with covide infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'Italy']   # Filter for country of your choice

country_covid_df.head()


In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''
country_covid__2020_df.head()

In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby


In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

# No need to add january in Italy
# country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

# country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby


In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 5002100  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('ITALY: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence_Stream%']
ax1.plot(months, ax1_ydata, label="Weighted Avg Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence_Stream%']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('ITALY: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence_Stream%")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()